IMPORT LIBRARIES

In [3]:
# pip install requests python-dotenv

import os
import requests
from datetime import datetime, timedelta
import time
from bs4 import BeautifulSoup as bs

MY STACK EXCHANGE API KEY

In [4]:
STACK_EXCHANGE_API_KEY = 'rl_2AK3hrozTLChbcAFfTMnbu2km'

In [5]:
element_protrusion_key_properties = ['position', 'float', 'height', 'min-height', 'width', 'min-width', 'display', 'margin', 'padding', 'font-size', 'white-space']
element_collision_key_properties = ['position', 'float', 'margin', 'height', 'min-height', 'width', 'min-width', 'display', 'margin', 'padding', 'flex-wrap', 'max-height', 'max-width']
viewport_protrusion_key_properties = ['position', 'float', 'height', 'min-height', 'width', 'min-width', 'margin', 'padding', 'font-size' 'white-space']
wrapping_key_properties = ['display', 'flex', 'float', 'width', 'min-width', 'margin', 'padding', 'font-size']

In [6]:
def keyword_search(properties_set, body):
    body_case_lower = body.lower()
    for keyword in properties_set:
        if keyword in body_case_lower:
             return True
    
    return False

In [10]:
from bs4 import NavigableString
import html

def clean_html_content(html_text):
    soup = bs(html_text, "lxml")
    cleaned = []
    
    for element in soup.descendants:
        if isinstance(element, NavigableString):
            if element.parent.name == 'code':
                # Preserve code block content
                decoded = html.unescape(str(element))
                cleaned.append(f'<code>{decoded}</code>')
            elif element.parent.name not in ['code', '[document]']:
                cleaned.append(' '.join(element.strip().split()))
    
    return '\n'.join([line for line in ' '.join(cleaned).split('\n') if line.strip()])

In [11]:
def fetch_stackoverflow_threads(tags, question, key_set, min_answer_count=1):
    page = 1
    has_more = True
    API_KEY = STACK_EXCHANGE_API_KEY
    base_url = "https://api.stackexchange.com/2.3/search/advanced"
    
    threads = []
    
    while has_more:
        params = {
            "page": page,
            "pagesize": 100,
            "order": "desc",
            "sort": "votes",
            "q": question,
            "tagged": ";".join(tags),
            "site": "stackoverflow",
            "key": API_KEY,
            "filter": "!0WRXThY5-I_TB(WxIdhBDpR5e"
        }
        
        try:
            response = requests.get(base_url, params=params)
            print(response.url)
            response.raise_for_status()
            data = response.json()
        
            for item in data.get("items", []):
                if item["answer_count"] >= min_answer_count:
                    thread = {
                        "title": item["title"],
                        "question_body": clean_html_content(item["body"]),
                        "link": item["link"],
                        "score": item["score"],
                        "tags": item["tags"],
                        "question_id": item["question_id"],
                        "answer_count": item["answer_count"],
                        "view_count": item["view_count"],
                        "comment_count": item["comment_count"],
                        "comments": [],
                        "answers": []
                    }

                    if item["comment_count"] > 0:
                        for comment in item["comments"]:
                            cleaned_text = clean_html_content(comment["body"])
                            if keyword_search(key_set, cleaned_text):
                                thread["comments"].append({"body": cleaned_text})

                    if item["is_answered"]:
                        answer_url = f'https://api.stackexchange.com/2.3/questions/{item["question_id"]}/answers'
                        params = {
                            "key": API_KEY,
                            'order': 'asc',
                            'sort': 'votes',    
                            'site': 'stackoverflow',   
                            'filter': '!)rlHR4RcMJ3iyACK5j)T',  
                        }
                        response = requests.get(answer_url, params=params)
                        print(response.url)
                        response.raise_for_status()
                        answers = response.json()

                        cleaned_answers = ''
                        for answer in answers['items']:
                            if answer['score'] > 0:
                                cleaned_answer = clean_html_content(answer["body"])
                                if keyword_search(key_set, cleaned_answer):
                                    thread["answers"].append({
                                        "score": answer["score"],
                                        "body": cleaned_answer
                                    })
                                
                if len(thread["answers"]) == 0 and len(thread["comments"]) == 0:
                    continue
                            
                threads.append(thread)

            # has_more = data.get('has_more', False)
            has_more = False
            page += 1

            # Respect rate limits
            if 'backoff' in response:
                time.sleep(response['backoff'] + 1)
    
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return []
        
    return threads

In [18]:
collision_tags = ['css', 'html']
question = "overlapping fix"
collision_qa_threads = fetch_stackoverflow_threads(
    tags=collision_tags,
    question=question,
    key_set=element_collision_key_properties,
    min_answer_count=1
)

https://api.stackexchange.com/2.3/search/advanced?page=1&pagesize=100&order=desc&sort=votes&q=overlapping+fix&tagged=css%3Bhtml&site=stackoverflow&key=rl_2AK3hrozTLChbcAFfTMnbu2km&filter=%210WRXThY5-I_TB%28WxIdhBDpR5e
https://api.stackexchange.com/2.3/questions/710158/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/8965063/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/3906640/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/21638859/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/13798446/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=v

C:\Users\ttaos\AppData\Local\Temp\ipykernel_23576\4204274084.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(html_text, "lxml")


https://api.stackexchange.com/2.3/questions/1900813/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/11059628/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/26826689/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/5894642/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/28627049/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/1702580/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3i

In [20]:
collision_tags = ['css', 'html']
question = "collision fix"
collision_qa_threads_v2 = fetch_stackoverflow_threads(
    tags=collision_tags,
    question=question,
    key_set=element_collision_key_properties,
    min_answer_count=1
)
collision_qa_threads.append(collision_qa_threads_v2)

https://api.stackexchange.com/2.3/search/advanced?page=1&pagesize=100&order=desc&sort=votes&q=collision+fix&tagged=css%3Bhtml&site=stackoverflow&key=rl_2AK3hrozTLChbcAFfTMnbu2km&filter=%210WRXThY5-I_TB%28WxIdhBDpR5e
https://api.stackexchange.com/2.3/questions/36764452/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/75216867/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/75860660/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/30448103/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/58283260/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort

In [23]:
protrusion_tags = ['css', 'html']
question = "overflow fix"
protrusion_qa_threads = fetch_stackoverflow_threads(
    tags=protrusion_tags,
    question=question,
    key_set=element_protrusion_key_properties,
    min_answer_count=1
)

https://api.stackexchange.com/2.3/search/advanced?page=1&pagesize=100&order=desc&sort=votes&q=overflow+fix&tagged=css%3Bhtml&site=stackoverflow&key=rl_2AK3hrozTLChbcAFfTMnbu2km&filter=%210WRXThY5-I_TB%28WxIdhBDpR5e
https://api.stackexchange.com/2.3/questions/306252/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/33454533/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/17408815/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/19794211/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/7289769/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=vot

C:\Users\ttaos\AppData\Local\Temp\ipykernel_23576\4204274084.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(html_text, "lxml")


https://api.stackexchange.com/2.3/questions/2403011/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/11520728/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/1461015/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/10056583/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/24193272/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/11855086/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3

In [25]:
wrapping_tags = ['css', 'html']
question = "element next line"
wrapping_qa_threads = fetch_stackoverflow_threads(
    tags=wrapping_tags,
    question=question,
    key_set=wrapping_key_properties,
    min_answer_count=1
)

https://api.stackexchange.com/2.3/search/advanced?page=1&pagesize=100&order=desc&sort=votes&q=element+next+line&tagged=css%3Bhtml&site=stackoverflow&key=rl_2AK3hrozTLChbcAFfTMnbu2km&filter=%210WRXThY5-I_TB%28WxIdhBDpR5e
https://api.stackexchange.com/2.3/questions/10272605/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/22310912/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/18339549/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/20695333/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/11151728/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&

C:\Users\ttaos\AppData\Local\Temp\ipykernel_23576\4204274084.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(html_text, "lxml")


https://api.stackexchange.com/2.3/questions/34681218/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/37659558/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/75024007/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/52556708/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/16246099/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/20639825/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcM

In [26]:
import json

def save_to_json(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
def save_to_file(threads, file_name):
    save_to_json(threads, file_name)
    print(f"Saved {len(threads)} threads to {file_name}")

In [27]:
save_to_file(collision_qa_threads, "D:\Codes\Try-LLM\stackoverflow_collision_threads.json")

Saved 92 threads to D:\Codes\Try-LLM\stackoverflow_collision_threads.json


In [28]:
save_to_file(protrusion_qa_threads, "D:\Codes\Try-LLM\stackoverflow_protrusion_threads.json")

Saved 87 threads to D:\Codes\Try-LLM\stackoverflow_protrusion_threads.json


In [29]:
save_to_file(wrapping_qa_threads, "D:\Codes\Try-LLM\stackoverflow_wrapping_threads.json")

Saved 87 threads to D:\Codes\Try-LLM\stackoverflow_wrapping_threads.json
